In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/ViT5/examples


In [2]:
import sys
sys.path.append("")

In [3]:
# !wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
# !unzip 'master.zip'

In [4]:
import glob
import pandas as pd
pd.set_option('display.max_columns', None)
import concurrent.futures
from datasets import *
import datasets
from sklearn.model_selection import train_test_split
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# from vncorenlp import VnCoreNLP
import torch 
import nltk
from nltk.tokenize import sent_tokenize



/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
OUTPUT_DIR = 'vietnamese_mt5_summary_model'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [6]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/mrzaizai2k/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Processing data

In [7]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

train_paths = listPaths('data/vietnews-master/data/train_tokenized/*')
val_paths = listPaths('data/vietnews-master/data/val_tokenized/*')
test_paths = listPaths('data/vietnews-master/data/test_tokenized/*')

In [8]:
def read_content(pathfile):
    """
    Input: Path of txt file
    Output: A dictionary has keys 'original' and 'summary'
    """
    with open(pathfile) as f:
      rows  = f.readlines()
      original = ' '.join(''.join(rows[4:]).split('\n'))
      summary = ' '.join(rows[2].split('\n'))
            
    return {'file' : pathfile,
              'original': original, 
              'summary': summary}

In [9]:
read_content(train_paths[0])

{'file': 'data/vietnews-master/data/train_tokenized/006157.txt.seg',
 'original': 'Tập 4 Thần_tượng Âm_nhạc nhí - Vietnam_Idol_Kids 2017 lên sóng tối 2/6 với màn tranh tài của top 8 thí_sinh nữ để lựa_chọn ra 5 tấm vé vào tiếp vòng trong . Trong đó , 3 giám_khảo Isaac - Văn_Mai_Hương - Bích_Phương được quyền lựa_chọn 3 thí_sinh . 2 thí_sinh còn lại sẽ do các giám_khảo khách mời quyết_định . Nữ ca_sĩ Văn_Mai_Hương xúc_động : “ Cô có may_mắn năm nay ngồi ở vị_trí ban giám_khảo , may_mắn hơn là được gặp con . Mỗi khi gặp Hiền , cô tự thấy bản_thân cô rất kém , bởi có lúc cô không trân_trọng cũng như không tin vào bản_thân ... Cảm_ơn con , bởi đôi_khi có những cái cô không bằng con được , đó là sự lạc_quan . Và cô tin , còn rất nhiều người phải học đức_tính lạc_quan này của Hiền . Con hát rất là hay ” . Đồng_tình với ý_kiến của đồng_nghiệp , Bích_Phương cũng xúc_động chia_sẻ : “ Giọt nước_mắt dành cho con là sự khâm_phục chứ không phải là thương_cảm . Cô chưa bao_giờ thấy con buồn . Từ lúc

In [10]:
def get_dataframe(pathfiles):
    with concurrent.futures.ProcessPoolExecutor() as executor:
      data = executor.map(read_content, pathfiles)
    
    # Make blank dataframe
    data_df = list()
    for d in data:
      data_df.append(d)
    data_df = pd.DataFrame(data_df)
    data_df.dropna(inplace = True)
    data_df = data_df.sample(frac=1).reset_index(drop=True)

    return data_df

In [11]:
train_df = get_dataframe(train_paths[:10000])

In [12]:
val_df = get_dataframe(val_paths[:10000])
test_df = get_dataframe(test_paths[:100])

In [13]:
test_df.head()

,file,original,summary
0,data/vietnews-master/data/test_tokenized/01796...,"Theo Hãng tin Bloomberg , trong bối_cảnh nhiều...",Nguồn_tin của hãng Bloomberg cho_biết cơ_quan_...
1,data/vietnews-master/data/test_tokenized/00547...,Hãng tin Reuters cho_biết đã đọc được 1 bản bá...,"Tổng_thống Robert_Mugabe từng là tượng_đài , l..."
2,data/vietnews-master/data/test_tokenized/01964...,"Tối 7/6 , Thành_uỷ Đà_Nẵng cho_biết đã chỉ_đạo...","Giám_đốc , phó giám_đốc Cảng vụ đường_thuỷ nội..."
3,data/vietnews-master/data/test_tokenized/01912...,"Ngày 28/3 , UBND huyện Hải_Lăng , Quảng_Trị cô...",9 con lợn của hộ dân ở xã Hải_Chánh ( Hải_Lăng...
4,data/vietnews-master/data/test_tokenized/00591...,' Hai chân ' ... bị lệch Thứ_trưởng Nguyễn_Văn...,Thứ_trưởng Bộ GD-ĐT Nguyễn_Văn_Phúc đã nhấn_mạ...


In [14]:
# test_df.to_parquet("data/vietnews/test.parquet", index=False)

In [15]:
# train_df = pd.read_parquet("data/vietnews/train.parquet")
# val_df = pd.read_parquet("data/vietnews/val.parquet")
# test_df = pd.read_parquet("data/vietnews/test.parquet")
# test_df.head()

In [16]:
# import matplotlib.pyplot as plt

# train_df['word_count'] = train_df['summary'].apply(lambda x: len(x.split()))

# # Plot the histogram
# plt.figure(figsize=(10, 6))
# plt.hist(train_df['word_count'], bins=100, edgecolor='black')
# plt.title('Histogram of Number of Words in Sentences')
# plt.xlabel('Number of Words')
# plt.ylabel('Frequency')
# plt.show()

## **Warm-starting RoBERTaShared for BBC XSum**

***Note***: This notebook only uses a few training, validation, and test data samples for demonstration purposes. To fine-tune an encoder-decoder model on the full training data, the user should change the training and data preprocessing parameters accordingly as highlighted by the comments.


### **Data Preprocessing**


In [17]:
# pretrained_model = "google/mt5-small"
pretrained_model = "huggingface-course/mt5-finetuned-amazon-en-es"
# facebook/bart-large-cnn
# "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)


In [18]:
train_data =  Dataset.from_pandas(train_df)
val_data =  Dataset.from_pandas(val_df)
test_data =  Dataset.from_pandas(test_df)

In [19]:
batch_size=16  # change to 16 for full training

max_input_length = 512 #1024
max_target_length = 128 #128


def process_data_to_model_inputs(batch):
    model_inputs = tokenizer(
        batch["original"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        batch["summary"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# only use 32 training examples for notebook - DELETE LINE FOR FULL TRAINING
# train_data = train_data.select(range(32))

train_data_batch = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file","original", "summary"],
)
train_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# val_data = val_data.select(range(16))

val_data_batch = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file", "original", "summary"],
)
val_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)

Map: 100%|██████████| 10000/10000 [00:05<00:00, 1769.79 examples/s]


In [20]:
tokenizer.pad_token_id

0

In [21]:
train_df.iloc[0]

file        data/vietnews-master/data/train_tokenized/0661...
original    Kết_quả xổ_số Vietlott loại_hình Power 6/55 ng...
summary     Sau khi " siêu độc_đắc " 300 tỷ nổ vào đầu thá...
Name: 0, dtype: object

In [22]:
a = tokenizer(str(train_df["summary"].iloc[0]))
print(str(train_df["summary"].iloc[0]))
a

Sau khi " siêu độc_đắc " 300 tỷ nổ vào đầu tháng 5 , nhiều người còn chưa “ hoàn_hồn ” vì thất_vọng thì những giải jackpot 2 liên_tiếp có người trúng thưởng . Hôm_nay là lần thứ 3 jackpot 2 tìm được chủ_nhân may_mắn sau khi jackpot 1 bị chinh_phục . 


{'input_ids': [7356, 2076, 313, 259, 263, 6121, 2214, 297, 290, 11095, 3625, 313, 2787, 259, 270, 8679, 259, 272, 1544, 259, 793, 268, 355, 1687, 259, 8800, 430, 259, 261, 677, 1990, 1672, 317, 3121, 562, 2294, 262, 359, 623, 2353, 290, 334, 8881, 259, 365, 259, 7568, 394, 1075, 290, 379, 6347, 394, 1135, 259, 272, 1992, 718, 1593, 60694, 356, 615, 1079, 290, 524, 1302, 325, 885, 1672, 534, 6927, 394, 5278, 259, 260, 447, 19698, 290, 16309, 796, 259, 280, 1313, 394, 1978, 381, 60694, 356, 259, 9164, 282, 259, 1318, 562, 708, 290, 272, 4912, 1432, 290, 282, 6701, 1520, 2076, 60694, 333, 330, 933, 259, 69036, 290, 6734, 2902, 259, 260, 259, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [23]:
tokenizer.convert_ids_to_tokens(train_data_batch[0]["labels"])

['▁Sau',
 '▁khi',
 '▁"',
 '▁',
 's',
 'iêu',
 '▁độ',
 'c',
 '_',
 'đ',
 'ắc',
 '▁"',
 '▁300',
 '▁',
 't',
 'ỷ',
 '▁',
 'n',
 'ổ',
 '▁',
 'và',
 'o',
 '▁đ',
 'ầu',
 '▁',
 'tháng',
 '▁5',
 '▁',
 ',',
 '▁nh',
 'iều',
 '▁người',
 '▁c',
 'òn',
 '▁ch',
 'ư',
 'a',
 '▁“',
 '▁ho',
 'àn',
 '_',
 'h',
 'ồn',
 '▁',
 '”',
 '▁',
 'vì',
 '▁th',
 'ất',
 '_',
 'v',
 'ọng',
 '▁th',
 'ì',
 '▁',
 'n',
 'hững',
 '▁gi',
 'ải',
 '▁jackpot',
 '▁2',
 '▁li',
 'ên',
 '_',
 'ti',
 'ế',
 'p',
 '▁có',
 '▁người',
 '▁tr',
 'úng',
 '▁th',
 'ưởng',
 '▁',
 '.',
 '▁H',
 'ôm',
 '_',
 'nay',
 '▁là',
 '▁',
 'l',
 'ần',
 '▁th',
 'ứ',
 '▁3',
 '▁jackpot',
 '▁2',
 '▁',
 'tì',
 'm',
 '▁',
 'được',
 '▁ch',
 'ủ',
 '_',
 'n',
 'hân',
 '▁may',
 '_',
 'm',
 'ắn',
 '▁sau',
 '▁khi',
 '▁jackpot',
 '▁1',
 '▁b',
 'ị',
 '▁',
 'chinh',
 '_',
 'ph',
 'ục',
 '▁',
 '.',
 '▁',
 '</s>']

In [24]:
# train_data_batch[0]

### **Warm-starting the Encoder-Decoder Model**

In [25]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model).to(device)

### **Fine-Tuning Warm-Started Encoder-Decoder Models**

The `Seq2SeqTrainer` that can be found under [examples/seq2seq/seq2seq_trainer.py](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/seq2seq_trainer.py) will be used to fine-tune a warm-started encoder-decoder model.

Let's download the `Seq2SeqTrainer` code and import the module along with `TrainingArguments`.

In [26]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

2024-06-09 19:03:26.180772: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-09 19:03:26.211471: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-09 19:03:26.829976: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


We need to add some additional parameters to make `TrainingArguments` compatible with the `Seq2SeqTrainer`. Let's just copy the `dataclass` arguments as defined in [this file](https://github.com/patrickvonplaten/transformers/blob/make_seq2seq_trainer_self_contained/examples/seq2seq/finetune_trainer.py).

Also, we need to define a function to correctly compute the ROUGE score during validation. ROUGE is a much better metric to track during training than only language modeling loss.

In [27]:
import numpy as np
import evaluate

rouge_score = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [28]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors='pt')

In [29]:
features = [train_data_batch[i] for i in range(2)]
data_collator(features)

/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


{'input_ids': tensor([[ 412, 1878,  290,  ..., 5182, 1013,    1],
        [ 447, 1725,  290,  ..., 2160,  300,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[ 7356,  2076,   313,   259,   263,  6121,  2214,   297,   290, 11095,
          3625,   313,  2787,   259,   270,  8679,   259,   272,  1544,   259,
           793,   268,   355,  1687,   259,  8800,   430,   259,   261,   677,
          1990,  1672,   317,  3121,   562,  2294,   262,   359,   623,  2353,
           290,   334,  8881,   259,   365,   259,  7568,   394,  1075,   290,
           379,  6347,   394,  1135,   259,   272,  1992,   718,  1593, 60694,
           356,   615,  1079,   290,   524,  1302,   325,   885,  1672,   534,
          6927,   394,  5278,   259,   260,   447, 19698,   290, 16309,   796,
           259,   280,  1313,   394,  1978,   381, 60694,   356,   259,  9164,
           282,   259,  1318,   562,   708,   290,   272,  4912,  1432,

Cool! Finally, we start training.

In [30]:
# set training arguments - these params are not really tuned, feel free to change
logging_steps = min(2000,len(train_data_batch) // batch_size)
print("logging_steps", logging_steps)

training_args = Seq2SeqTrainingArguments(
    output_dir= OUTPUT_DIR,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    eval_strategy="epoch",
    learning_rate=1e-3,
    # do_train=True,
    # do_eval=True,
    logging_steps=logging_steps,  # set to 2000 for full training
    save_steps=500,  # set to 500 for full training
    eval_steps=75,  # set to 7500 for full training
    warmup_steps=3000,  # set to 3000 for full training
    num_train_epochs=10, #uncomment for full training
    overwrite_output_dir=True,
    optim='adafactor',
    save_total_limit=3,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model,
    args=training_args,
    train_dataset=train_data_batch,
    eval_dataset=val_data_batch,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

logging_steps 625


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.778100,1.839109,28.502400,11.271700,22.108000,22.436100
2,2.062200,1.757558,28.024500,10.611200,21.735300,22.068500
3,1.863600,1.617907,27.353000,10.623800,21.468600,21.751200
4,1.740800,1.614288,28.092800,11.285700,22.060000,22.362900
5,1.649200,1.541136,27.820900,10.918400,21.681900,21.977300
6,1.544800,1.480204,28.043300,11.423200,22.069600,22.373000
7,1.445400,1.462143,27.855200,11.170800,21.895800,22.194900
8,1.363600,1.452241,28.326400,11.794500,22.356300,22.652400
9,1.297800,1.434685,28.444000,11.938800,22.427900,22.734400
10,1.244500,1.444718,28.620500,12.189200,22.662600,22.963500


/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/g

TrainOutput(global_step=6250, training_loss=1.69900193359375, metrics={'train_runtime': 39490.375, 'train_samples_per_second': 2.532, 'train_steps_per_second': 0.158, 'total_flos': 5.2874969088e+16, 'train_loss': 1.69900193359375, 'epoch': 10.0})

In [31]:
trainer.save_model(OUTPUT_DIR)

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

events.out.tfevents.1717934610.DESKTOP-H2CRQMR.662540.0: 100%|██████████| 12.4k/12.4k [00:00<00:00, 21.3kB/s]
model.safetensors: 100%|██████████| 1.20G/1.20G [01:33<00:00, 12.9MB/s]
Upload 2 LFS files: 100%|██████████| 2/2 [01:33<00:00, 46.93s/it]


In [32]:
# !gsutil -m cp -r '/content/training/*' 'gs://kaggle-vbdi-test/training_Data'

### **Evaluation**

Awesome, we finished training our dummy model. Let's now evaluated the model on the test data. We make use of the dataset's handy `.map()` function to generate a summary of each sample of the test data.

In [41]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model, use_fast=False)

# model = EncoderDecoderModel.from_pretrained(OUTPUT_DIR + "/checkpoint-4000")
model = AutoModelForSeq2SeqLM.from_pretrained("my_fine_tuned_t5_small_model")

model.to("cuda")

# test_data = datasets.load_dataset("xsum", split="test")

batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=256, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask,
                             max_length=256,
                        num_beams=5,
                        no_repeat_ngram_size=2,
                        early_stopping=True,)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["original"])

pred_str = results["pred"]
label_str = results["summary"]

Map: 100%|██████████| 100/100 [05:03<00:00,  3.03s/ examples]


In [42]:
rouge_output = rouge_score.compute(predictions=pred_str, references=label_str, use_stemmer=True)
rouge_output

{'rouge1': 0.46635404475648523,
 'rouge2': 0.17445885772888028,
 'rougeL': 0.3090386583457608,
 'rougeLsum': 0.3091414947465033}

In [43]:
for key,value in rouge_output.items():
  print(key)
  print(value)

rouge1
0.46635404475648523
rouge2
0.17445885772888028
rougeL
0.3090386583457608
rougeLsum
0.3091414947465033


In [47]:
for i in range(10):
    print('\nPrediction: ',pred_str[i])
    print('Truth: ',label_str[i])
    print('Content: ',test_data[i]['original'])
    


Prediction:  Dự_án cải_tạo Bến xe_buýt Tân_Quy với kinh_phí 1,9 tỷ đồng.
Truth:  Trung_tâm Quản_lý giao_thông công_cộng TP. HCM đưa vào sử_dụng 2 bến xe_buýt mới là bến xe_buýt Tân_Quy ( xã Trung_An - Củ_Chi ) và bến xe_buýt An_Nhơn_Tây ( xã An_Nhơn_Tây - Củ_Chi ) phục_vụ người_dân đi_lại dịp lễ . 
Content:  Dự_án cải_tạo Bến xe_buýt Tân_Quy với kinh_phí 1,9 tỷ đồng , công_trình có tổng diện_tích xây_dựng 1.156 m2 tại gần giao_lộ đường_Tỉnh_Lộ 8 và đường Sen_Hồng , thuộc xã Trung_An ( huyện Củ_Chi ) . Tại bến_xe xây_dựng mới 1 nhà_vệ_sinh công_cộng , nhà điều_hành , nhà bảo_vệ và xây_dựng hàng_rào bảo_vệ , hệ_thống chiếu sáng nhằm phục_vụ tốt hơn nhu_cầu đi_lại bằng xe_buýt của người_dân . Tuyến xe_buýt qua bến này là tuyến 122 ( Bến_xe An_Sương - Tân_Quy ) và tuyến 70 ( Tân_Quy - Bến_Súc ) . Dự_án Xây_dựng bến xe_buýt An_Nhơn_Tây có tổng diện_tích xây_dựng 600m2 tại đường An_Nhơn_Tây , thuộc xã An_Nhơn_Tây ( huyện Củ_Chi ) . Bến_xe này được xây_dựng với đầy_đủ trang_thiết_bị , nhà_vệ

In [45]:
test_data[0]['file']

'data/vietnews-master/data/test_tokenized/003528.txt.seg'